---
layout: post
title: Sprint 5 Review
description: Guess 
author: Keerthan Karumudi
type: issue
comments: true
---

# Sprint 5 Review

## Our Project

### Group
The purpose of our group’s program is to create a multiplayer drawing and guessing game inspired by Scribble.io. This program allows players to take turns drawing while others guess the word or phrase being illustrated. The goal is to provide an entertaining and interactive way for people to engage in creative gameplay, fostering collaboration, quick thinking, and creativity.


### Individual
My individual feature focuses on implementing the real-time guessing system. This feature enables players to submit guesses while others draw, providing instant feedback on whether the guess is correct. The purpose of this feature is to ensure smooth and engaging gameplay by allowing seamless interaction between players and keeping the game dynamic and competitive.

## FullStack Features

### Frontend api request and response

In [ ]:
document.getElementById('guessForm').addEventListener('submit', async (e) => {
    e.preventDefault();
  
    const user = document.getElementById('user').value.trim();
    const guess = document.getElementById('guess').value.trim();
    const is_correct = guess.toLowerCase() === currentDrawing.label.toLowerCase();
  
    const guessData = { user, guess, is_correct };
  
    try {
      console.log('Sending guess data:', guessData); // Log the guess data being sent
  
      // API Request to submit the guess
      const response = await fetch('http://127.0.0.1:8887/api/submit_guess', {
        method: 'POST',
        headers: {
          'Content-Type': 'application/json',
        },
        body: JSON.stringify(guessData),
      });
  
      if (!response.ok) {
        throw new Error('Network response was not ok');
      }
  
      // Handling the API Response
      const result = await response.json();
      console.log('Response from server:', result); // Log the server response
  
      messageArea.textContent = result.message;
  
      // Update the guess list based on response
      if (is_correct) {
        guessList.innerHTML += `<li><strong>${user}</strong> guessed correctly: ${guess}</li>`;
      } else {
        guessList.innerHTML += `<li><strong>${user}</strong> guessed incorrectly: ${guess}</li>`;
      }
    } catch (error) {
      console.error('Error submitting guess:', error);
      messageArea.textContent = `There was an error submitting your guess. Please try again. Error: ${error.message}`;
    }
  });  

### Postman Raw API Request and Restful response

Method POST

JSON Body:

{
  "user": "Keerthan",
  "guess": "ocean",
  "is_correct": true
}

JSON Response: 201 Created

{
    "Latest Guess": {
        "Guess": "ocean",
        "Is Correct": true
    },
    "Stats": {
        "Correct Guesses": 1,
        "Total Guesses": 1,
        "Wrong Guesses": 0
    },
    "User": "Keerthan"
}



### db_init, restore, and backup Data Recovery

## List Requests

In our program, we make use of lists, dictionaries, and a database to handle the application's data. Specifically:

- Lists are used to store rows of guesses 
- Dictionaries are used to handle the JSON api requests and responses
- Database (SQLite) is used to store the user data(the guesses)

### Frontend portion for Formatting Response Data (JSON) from API into DOM:


// Add an event listener to handle form submission
document.getElementById('guessForm').addEventListener('submit', async (e) => {
  e.preventDefault(); // Prevent the form from refreshing the page on submission

  // Retrieve user input values from the form
  const user = document.getElementById('user').value.trim(); // Get the user's name
  const guess = document.getElementById('guess').value.trim(); // Get the user's guess
  const is_correct = guess.toLowerCase() === currentDrawing.label.toLowerCase(); // Check if the guess matches the current drawing's label

  // Create an object with the user's guess data
  const guessData = { user, guess, is_correct };

  try {
    // Send the guess data to the API using a POST request
    const response = await fetch('http://127.0.0.1:8887/api/submit_guess', {
      method: 'POST', // HTTP method
      headers: {
        'Content-Type': 'application/json', // Specify JSON data format
      },
      body: JSON.stringify(guessData), // Convert the data object to JSON format
    });

    // Check if the response is not okay and throw an error
    if (!response.ok) {
      throw new Error('Network response was not ok'); // Handle failed responses
    }

    // Parse the JSON response from the API
    const result = await response.json();

    // Get references to DOM elements for displaying messages and guesses
    const messageArea = document.getElementById('messageArea'); // Element to display feedback
    const guessList = document.getElementById('guessList'); // List to show all guesses

    // Display the message returned from the server
    messageArea.textContent = result.message; // Update the message area with server feedback

    // Create a new list item to display the user's guess
    const newGuess = document.createElement('li'); // Create a new list item
    newGuess.innerHTML = is_correct
      ? `<strong>${user}</strong> guessed correctly: ${guess}` // Format for a correct guess
      : `<strong>${user}</strong> guessed incorrectly: ${guess}`; // Format for an incorrect guess

    // Append the new list item to the guess list
    guessList.appendChild(newGuess);

  } catch (error) {
    // Log any errors and display an error message to the user
    console.error('Error submitting guess:', error); // Log error details for debugging
    document.getElementById('messageArea').textContent = 'Error submitting guess. Please try again.'; // Notify the user of the error
  }
});


### Database queries

In [ ]:
@app.route('/api/submit_guess', methods=['GET'])
def get_guesses():
    print("GET /api/guesses route was hit")  # Debugging line

    try:
        # Extract the 'user' query parameter
        user = request.args.get('user', type=str)

        # If 'user' is provided, return all guesses for that user
        if user:
            guesses = Guess.query.filter_by(guesser_name=user).all()  # Fetch guesses for the given user
            if not guesses:
                return jsonify({"error": "No guesses found for this user."}), 404
            # Return all guesses for the specified user
            return jsonify([guess.read() for guess in guesses]), 200

        # If 'user' is not provided, return an error
        else:
            return jsonify({"error": "Please provide the 'user' as a query parameter."}), 400

    except Exception as e:
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500


### Methods in Class to Work with Columns:

In [ ]:
@app.route('/api/submit_guess', methods=['POST'])
def save_guess_simple():
    try:
        # Parse JSON input
        data = request.json
        if not data:
            return jsonify({"error": "Invalid or missing JSON payload."}), 400

        print("Incoming request data:", data)  # Debugging

        required_keys = {'user', 'guess', 'is_correct'}
        is_valid, error_message = validate_request_data(data, required_keys)

        # Validate input data
        if not is_valid:
            print("Validation failed:", error_message)  # Debugging
            return jsonify({"error": error_message}), 400

        # Extract values from the request
        user = data['user']
        guess = data['guess']
        is_correct = data['is_correct']

        # Ensure the types of the inputs are correct
        if not isinstance(user, str) or not isinstance(guess, str) or not isinstance(is_correct, bool):
            return jsonify({"error": "Invalid data types for user, guess, or is_correct."}), 400

        # Initialize stats for the user if not present
        if user not in user_stats:
            user_stats[user] = {
                "correct": 0,
                "wrong": 0,
                "total_guesses": 0,
                "guesses": []
            }

        # Update user stats
        user_stats[user]["total_guesses"] += 1
        if is_correct:
            user_stats[user]["correct"] += 1
        else:
            user_stats[user]["wrong"] += 1

        # Append guess details to the user's history
        user_stats[user]["guesses"].append({
                "guess": guess,
                "is_correct": is_correct
            })

        # Append new guess to global chat logs
        chat_logs.append({
            "user": user,
            "guess": guess,
            "is_correct": is_correct
        })

        # Append new guess to the database
        try:
            # Initialize the database table
            initGuessDataTable()
            new_guess = Guess(
                guesser_name=user,
                guess=guess,
                is_correct=is_correct
            )

            db.session.add(new_guess)
            db.session.commit()  # Save to the database
            print("Guess saved to database successfully.")  # Debugging
        except Exception as e:
            print(f"Error saving guess to database: {e}")
            db.session.rollback()  # Roll back the transaction on failure
            return jsonify({"error": "Failed to save guess to database."}), 500

        # Prepare the response
        response_data = {
            "User": user,
            "Stats": {
                "Correct Guesses": user_stats[user]["correct"],
                "Wrong Guesses": user_stats[user]["wrong"],
                "Total Guesses": user_stats[user]["total_guesses"]
            },
            "Latest Guess": {
                "Guess": guess,
                "Is Correct": is_correct
            }
        }
        # Return success response with stats and latest guess
        return jsonify(response_data), 201

    except Exception as e:
        # Log unexpected exceptions and provide detailed debugging information
        print("General Exception:", str(e))
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500


## Algorithmic code request.

### Definition of Code Blocks to Handle Requests

Api CRUD Methods:

In [ ]:
@app.route('/api/submit_guess', methods=['POST'])
def save_guess_simple():
    try:
        # Parse JSON input
        data = request.json
        if not data:
            return jsonify({"error": "Invalid or missing JSON payload."}), 400

        print("Incoming request data:", data)  # Debugging

        required_keys = {'user', 'guess', 'is_correct'}
        is_valid, error_message = validate_request_data(data, required_keys)

        # Validate input data
        if not is_valid:
            print("Validation failed:", error_message)  # Debugging
            return jsonify({"error": error_message}), 400

        # Extract values from the request
        user = data['user']
        guess = data['guess']
        is_correct = data['is_correct']

        # Ensure the types of the inputs are correct
        if not isinstance(user, str) or not isinstance(guess, str) or not isinstance(is_correct, bool):
            return jsonify({"error": "Invalid data types for user, guess, or is_correct."}), 400

        # Initialize stats for the user if not present
        if user not in user_stats:
            user_stats[user] = {
                "correct": 0,
                "wrong": 0,
                "total_guesses": 0,
                "guesses": []
            }

        # Update user stats
        user_stats[user]["total_guesses"] += 1
        if is_correct:
            user_stats[user]["correct"] += 1
        else:
            user_stats[user]["wrong"] += 1

        # Append guess details to the user's history
        user_stats[user]["guesses"].append({
                "guess": guess,
                "is_correct": is_correct
            })

        # Append new guess to global chat logs
        chat_logs.append({
            "user": user,
            "guess": guess,
            "is_correct": is_correct
        })

        # Append new guess to the database
        try:
            # Initialize the database table
            initGuessDataTable()
            new_guess = Guess(
                guesser_name=user,
                guess=guess,
                is_correct=is_correct
            )

            db.session.add(new_guess)
            db.session.commit()  # Save to the database
            print("Guess saved to database successfully.")  # Debugging
        except Exception as e:
            print(f"Error saving guess to database: {e}")
            db.session.rollback()  # Roll back the transaction on failure
            return jsonify({"error": "Failed to save guess to database."}), 500

        # Prepare the response
        response_data = {
            "User": user,
            "Stats": {
                "Correct Guesses": user_stats[user]["correct"],
                "Wrong Guesses": user_stats[user]["wrong"],
                "Total Guesses": user_stats[user]["total_guesses"]
            },
            "Latest Guess": {
                "Guess": guess,
                "Is Correct": is_correct
            }
        }
        # Return success response with stats and latest guess
        return jsonify(response_data), 201

    except Exception as e:
        # Log unexpected exceptions and provide detailed debugging information
        print("General Exception:", str(e))
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500


@app.route('/api/submit_guess', methods=['GET'])
def get_guesses():
    print("GET /api/guesses route was hit")  # Debugging line

    try:
        # Extract the 'user' query parameter
        user = request.args.get('user', type=str)

        # If 'user' is provided, return all guesses for that user
        if user:
            guesses = Guess.query.filter_by(guesser_name=user).all()  # Fetch guesses for the given user
            if not guesses:
                return jsonify({"error": "No guesses found for this user."}), 404
            # Return all guesses for the specified user
            return jsonify([guess.read() for guess in guesses]), 200

        # If 'user' is not provided, return an error
        else:
            return jsonify({"error": "Please provide the 'user' as a query parameter."}), 400

    except Exception as e:
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500




@app.route('/api/submit_guess', methods=['PUT'])
def update_guess():
    try:
        # Parse JSON payload
        data = request.json
        if not data:
            return jsonify({"error": "Invalid or missing JSON payload."}), 400

        # Ensure that the required fields are present in the data
        required_keys = ['user', 'guess', 'is_correct']
        for key in required_keys:
            if key not in data:
                return jsonify({"error": f"Missing required field: {key}"}), 400

        # Get the existing guess by user (or another identifier you prefer)
        guess = Guess.query.filter_by(guesser_name=data['user']).first()
        if not guess:
            return jsonify({"error": "Guess not found for this user."}), 404

        # Update the guess fields
        guess.guess = data['guess']
        guess.is_correct = data['is_correct']

        # Commit the changes to the database
        db.session.commit()

        # Return a success response with the updated guess data
        return jsonify({
            "message": "Guess updated successfully.",
            "updated_data": guess.read()
        }), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500



@app.route('/api/submit_guess', methods=['DELETE'])
def delete_guess():
    try:
        # Parse JSON payload to get the guesser_name and guess
        data = request.json
        if not data or 'guesser_name' not in data or 'guess' not in data:
            return jsonify({"error": "Invalid or missing JSON payload. 'guesser_name' and 'guess' are required."}), 400

        guesser_name = data['guesser_name']
        guess = data['guess']
        
        # Find the guess in the database based on guesser_name and guess
        guess_entry = Guess.query.filter_by(guesser_name=guesser_name, guess=guess).first()
        if not guess_entry:
            return jsonify({"error": "Guess not found."}), 404

        # Delete the guess entry
        guess_entry.delete()
        
        # Commit the deletion to the database
        db.session.commit()

        return jsonify({"message": f"Guess for user '{guesser_name}' with guess '{guess}' deleted successfully."}), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500

### Method/Procedure in Class with Sequencing, Selection, and Iteration

In [ ]:
@app.route('/api/submit_guess', methods=['DELETE'])
def delete_guess():
    try:
        # Parse JSON payload to get the guesser_name and guess
        data = request.json
        if not data or 'guesser_name' not in data or 'guess' not in data:
            return jsonify({"error": "Invalid or missing JSON payload. 'guesser_name' and 'guess' are required."}), 400

        guesser_name = data['guesser_name']
        guess = data['guess']
        
        # Find the guess in the database based on guesser_name and guess
        guess_entry = Guess.query.filter_by(guesser_name=guesser_name, guess=guess).first()
        if not guess_entry:
            return jsonify({"error": "Guess not found."}), 404

        # Delete the guess entry
        guess_entry.delete()
        
        # Commit the deletion to the database
        db.session.commit()

        return jsonify({"message": f"Guess for user '{guesser_name}' with guess '{guess}' deleted successfully."}), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500

### Parameters and Return Type

- Parameters
    The body of the request is in JSON format, which contains fields like:

        guesser_name: Name of the guesser (string).
        guess: The guessed value (string).
        is_correct: Whether the guess is correct (boolean).

- Return Type
The functions return a JSON response, created using jsonify(), which ensures the response is in proper JSON format. 
     
     {
    "id": 1,
    "guesser_name": "JohnDoe",
    "guess": "apple",
    "is_correct": true
}


## Call to Algorithm request

### Definition of Code Block to Make a Request

Frontend Fetch to Endpoint:

const getGuessesForUser = async (username) => {
  try {
    const response = await fetch(`/api/guesses?user=${username}`, {
      method: 'GET',
      headers: {
        'Content-Type': 'application/json',
      },
    });

    if (response.ok) {
      const data = await response.json();
      console.log("User's guesses:", data);
      return data;
    } else {
      const errorData = await response.json();
      console.error('Error fetching guesses:', errorData);
      return errorData;
    }
  } catch (error) {
    console.error('Network or server error:', error);
  }
};

### Discuss the Call/Request to the Method with Algorithim

- Call/Request:
    - The delete CRUD method sends delete request to api/submit_guess endpoint for guess deletion

- Return/Response:
    - The response from the backend is handled by checking the status code and updating the UI  accordingly.

### Handling Data and Error Conditions

- Normal Conditions:
    - The interest is successfully deleted, and the UI is updated to reflect the change.

- Error Conditions:
    - If the interest is not found or the request fails, an error message is displayed.